In [24]:
import importlib
from pathlib import Path

import instructor
import openai
import pandas as pd

from cuery import pprint, response, task, utils
from cuery.work import CountrySectors, DirceJobs, DirceTasks, SpanishSectors

GDRIVE = Path("~/Library/CloudStorage/GoogleDrive-victoriano@graphext.com/Shared drives/Solutions").expanduser()
task.Task.registry

{'SpanishSectors': <cuery.task.Task at 0x11b4d7150>,
 'CountrySectors': <cuery.task.Task at 0x11d4e6850>,
 'DirceJobs': <cuery.task.Task at 0x11b5b0510>,
 'DirceTasks': <cuery.task.Task at 0x11b5b31d0>}

In [25]:
from pydantic import Field
from enum import Enum
from cuery.prompt import Prompt
from cuery.task import Task

from cuery.response import ResponseModel

class SimpleSectors(ResponseModel):
    sectors: list[str] = Field(
        description="A list of 5 NAIC industrial sectors with their AI automation potential",
        min_length=5,
        max_length=5,
    )

prompt = Prompt(messages=[
    {"role": "user", "content": """ Please identify sectors of industries/companies in {{country}} that have great potential to be disrupted
        by AI (using AI software automation). Use the North American Industry Classification System (NAICS) to identify
        the NAMES of sector and subsector (not IDs). Focus on sectors with the biggest potential for disruption, employing a large
        number of jobs, and having the biggest number of companies."""}],
    required=["country"]
)

CountrySectors = Task(
    name="CountrySectors", prompt=prompt, response=SimpleSectors
)

In [26]:
context = [{"country": "Unites States"}, {"country": "Spain"}]
sectors_simple = await CountrySectors(context, n_concurrent=20)
sectors_simple

Gathering responses: 100%|██████████| 2/2 [00:01<00:00,  1.51it/s]


[SimpleSectors(sectors=['Professional, Scientific, and Technical Services', 'Health Care and Social Assistance', 'Manufacturing', 'Retail Trade', 'Finance and Insurance']), SimpleSectors(sectors=['Professional, Scientific, and Technical Services', 'Manufacturing', 'Wholesale Trade', 'Retail Trade', 'Real Estate and Rental and Leasing'])]

In [27]:
sectors_simple.to_pandas()

,country,sectors
0,Unites States,"Professional, Scientific, and Technical Services"
1,Unites States,Health Care and Social Assistance
2,Unites States,Manufacturing
3,Unites States,Retail Trade
4,Unites States,Finance and Insurance
5,Spain,"Professional, Scientific, and Technical Services"
6,Spain,Manufacturing
7,Spain,Wholesale Trade
8,Spain,Retail Trade
9,Spain,Real Estate and Rental and Leasing


In [28]:
DATA_DIR = GDRIVE / "Research/future_of_work/inputs/ine_dirce_aggregated_by_activity.parquet"
data = pd.read_parquet(DATA_DIR)
data.columns = [c.replace(" ", "_") for c in data.columns]
data = data.rename(columns={"Division": "sector", "Actividad_principal": "subsector"})
display(data)

n_examples = 5
df = data.iloc[:5]

FileNotFoundError: [Errno 2] No such file or directory: '/Users/victoriano/Library/CloudStorage/GoogleDrive-victoriano@graphext.com/Research/future_of_work/inputs/ine_dirce_aggregated_by_activity.parquet'